In [ ]:
pip install aspect-based-sentiment-analysis

In [ ]:
import aspect_based_sentiment_analysis as absa

nlp = absa.load()

In [5]:
import warnings
warnings.filterwarnings("ignore")
from google.colab import files
import pandas as pd
import numpy as np

# Sentiment - delivery

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
path ='/content/drive/MyDrive/5153/meta_and_review.csv'
df = pd.read_csv(path)

In [ ]:
df.dropna(subset=['review'], axis=0, inplace=True)

In [ ]:
df.shape

(133159, 13)

In [ ]:
text_df = df[['review']]
text_df['Ori_index'] = text_df.index

In [ ]:
text_df.reset_index(drop=True, inplace=True)
text_df.head(-5)

,review,Ori_index
0,this was a good buy great shampoo and conditio...,0
1,i have used kms in the past and always loved t...,1
2,i remember kms products being a prestige salon...,2
3,this is a wonderful shampoo doesnt make your h...,3
4,i have a very dry frizzy hair i have tried man...,4
...,...,...
133149,i use this shampoo along with another product ...,133156
133150,so effective within a few weeks after using th...,133157
133151,my hairdresser told me that there were natural...,133158
133152,i have never tried shea butter before but thou...,133159


### filter our reviews which contains deivery_aspect

In [ ]:
delivery_aspect = ['delivery', 'logistics', 'shipment', 'transportation', 'transport']
delivery_list =[]
for i in range(len(delivery_aspect)):
  temp_list = list(text_df[text_df.review.str.contains(delivery_aspect[i])].index) 
  delivery_list = delivery_list + temp_list
delivery_idx= list(set(delivery_list))

In [ ]:
with pd.option_context ('display.max_colwidth', None):
  display(text_df[text_df.review.str.contains('support')].review)

145                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     i was so very disappointed with my purchase of the avalon thickening shampoo and conditioner i had such high hopes but in only two shampooings my hair was notic

In [ ]:
delivery_idx = sorted(delivery_idx)

In [ ]:
long_text_512_idx_list = list(text_df[text_df.loc[:,'review'].str.len() >=1999].review.index)

In [ ]:
delivery_index_removed = [x  for x in delivery_idx if x in long_text_512_idx_list]
len(delivery_index_removed)

13

In [ ]:
#update delivery index
#delivery_idx.remove(delivery_index_removed)
#remove extremel long text which this package can not handle
delivery_idx =[x for x in delivery_idx if x not in delivery_index_removed]

In [ ]:
len(delivery_idx)

1243

In [ ]:
n = 350
for idx in delivery_idx[350:450]:
  if len(text_df.review.iloc[idx]) >1900:
    print(n, len(text_df.review.iloc[idx]))
  n+=1

356 1901
404 1959


In [ ]:
nlp(text_df.review.iloc[404], aspects=['delivery','shipment'])

CompletedTask(text='dont be fooled because this conditioner is not expensive  this is one of the best conditioners i have ever used and i have paid for some of the best  neutrogena clean conditoner rinses clean so your hair feels nice and clean and full of body and you only need to use the smallest amount', aspects=['delivery', 'shipment'], subtasks=OrderedDict([('delivery', CompletedSubTask(text='dont be fooled because this conditioner is not expensive  this is one of the best conditioners i have ever used and i have paid for some of the best  neutrogena clean conditoner rinses clean so your hair feels nice and clean and full of body and you only need to use the smallest amount', aspect='delivery', examples=[PredictedExample(text='dont be fooled because this conditioner is not expensive  this is one of the best conditioners i have ever used and i have paid for some of the best  neutrogena clean conditoner rinses clean so your hair feels nice and clean and full of body and you only nee

In [ ]:
len(delivery_idx)

1243

In [ ]:
for idx in delivery_idx[:1243]: 
  delivery, shipment = nlp(text_df.review.iloc[idx], aspects=['delivery','shipment'])
  text_df.loc[idx, 'neu_delivery'] = delivery.scores[0]
  text_df.loc[idx,'neg_delivery'] = delivery.scores[1]
  text_df.loc[idx,'pos_delivery'] = delivery.scores[2]
  text_df.loc[idx, 'neu_shipment'] = shipment.scores[0]
  text_df.loc[idx,'neg_shipment'] = shipment.scores[1]
  text_df.loc[idx,'pos_shipment'] = shipment.scores[2]

In [ ]:
# impute na by zero 
text_df = text_df.fillna(0)

In [ ]:
text_df.head()

,review,Ori_index,neu_delivery,neg_delivery,pos_delivery,neu_shipment,neg_shipment,pos_shipment
0,this was a good buy great shampoo and conditio...,0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
1,i have used kms in the past and always loved t...,1,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
2,i remember kms products being a prestige salon...,2,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
3,this is a wonderful shampoo doesnt make your h...,3,0.002123,0.001227,0.99665,0.148797,0.167336,0.683867
4,i have a very dry frizzy hair i have tried man...,4,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000


In [ ]:
#overall sentiment score output
#merge dataset
path2= '/content/drive/MyDrive/5153/meta_and_review_Overall_sentiment.csv'
df_sentiment = pd.read_csv(path2)

In [ ]:
df_sentiment.dropna(subset=['review'], axis=0, inplace=True)

In [ ]:
#df na removed
print(df.shape, df_sentiment.shape)

(133159, 13) (133159, 17)


In [ ]:
df_full = df_sentiment.merge(text_df, how = 'left', left_index=True, right_on= 'Ori_index')

In [ ]:
df_full.rename(columns = {'review_x':'review'},inplace=True)
df_full.drop(['Ori_index', 'review_y'], axis=1, inplace=True)

In [ ]:
with pd.option_context ('display.max_colwidth', None):
  display(df_full.iloc[:5].review)

0                                                                                                                                                                                                                                         this was a good buy great shampoo and conditioner im pregnant right now and for some reason by hairs really dry and these products did the trick
1                                                                                                                                                                                                                                                                           i have used kms in the past and always loved their products this time around im not sure if i liked it as much
2                                                             i remember kms products being a prestige salon brand from the late 70s early 80s then they seemed to disappear  i use this as one of the rotating conditioners after washing vinegar

In [ ]:
df_full.loc[:, 'review_len'] = df_full.review.str.len()

In [ ]:
df_full.head()
print(df_full.shape)

(133159, 24)


In [ ]:
df_full.to_csv('meta_and_review_sentiment_aspect_delivery.csv', index=None)
files.download('meta_and_review_sentiment_aspect_delivery.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>